# Imviz Demonstration Notebook
## Roman/ASDF Edition

This notebook demonstrates the Imviz API in the Notebook setting when using simulated observations from The Nancy Grace Roman Space Telescope. UI equivalents for these actions, as well as additional documentation about Imviz, can be found here: https://jdaviz.readthedocs.io/en/latest/imviz/

Import modules needed for this notebook.

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table

from jdaviz import Imviz

We also need this to display Matplotlib in the notebook later.

In [ ]:
%matplotlib inline

For now, we will use synthetic data for testing. 

Before downloading the example data, which were generated with schema from `roman_datamodels` and `rad == 0.22.0`, check that the correct version of `roman_datamodels` is installed. Otherwise it may not be possible to open these example files:

In [ ]:
from packaging.version import Version
from astropy.utils.data import download_file
from roman_datamodels import __version__ as rdm_version

assert Version(rdm_version) >= Version('0.22.0'), 'Update roman_datamodels with `pip install -U roman_datamodels`'


# Each of these files is ~1 GB:
image_model1 = download_file('https://stsci.box.com/shared/static/oor05ut2u5e3xq4kcd244pq5md1el13h.asdf', cache=True)
image_model2 = download_file('https://stsci.box.com/shared/static/sw1jvu70q20w6czi8pmd9351f0yqkj5o.asdf', cache=True)

In [ ]:
# load the observations into the helper
imviz = Imviz()

with warnings.catch_warnings():
    warnings.simplefilter('ignore')  # ErfaWarning
    imviz.load_data(image_model1, ext='data', data_label='exposure 1')
    imviz.load_data(image_model2, ext='data', data_label='exposure 2')

Then, we visualize the data and start off by looking at some of the basic features:

In [ ]:
imviz.show('sidecar:split-right', height=1000)

We have loaded two sequential exposures with the same pointing. Let's orient both images by their WCS:

In [ ]:
orientation = imviz.plugins['Orientation']

orientation.align_by = 'WCS'

Imviz will initially display Roman images zoomed to fit the display. However, since each detector's images have 4k resolution, the default Imviz zoom makes it hard to tell if the colormap stretch is appropriate.

Panning and zooming is possible by showing the viewer toolbar and clicking on the '+'-shaped icon, then dragging around in the image and using scrolling to zoom in and out. To change the stretch and colormap, show the **Layer** options accessible through the last icon in the viewer toolbar.

We can also change these programmatically, for example the stretch:

In [ ]:
plot_options = imviz.plugins['Plot Options']

for viewer_name in imviz.app.get_viewer_reference_names():
    viewer = imviz.app.get_viewer(viewer_name)
    plot_options.viewer = viewer_name
    
    for data in viewer.data():
        plot_options.layer = data.label
        
        # set colormap and stretch for this layer:
        plot_options.stretch_function = 'arcsinh'
        plot_options.image_colormap = 'Viridis'
        plot_options.stretch_vmin = 0
        plot_options.stretch_vmax = 20

Note also that in the above example there are mouse-over coordinates visible by default.

It possible to make selections/regions in images and export these to Astropy regions. Click on the viewer toolbar then click on the circular selection tool, and drag and click to select an interesting region on the sky. We can then export this region with:

In [ ]:
regions = imviz.get_interactive_regions()

In [ ]:
regions

Since the region is an Astropy region, we can e.g. convert it to a mask:

In [ ]:
wcs = imviz.app.data_collection[-1].coords
mask = regions['Subset 1'].to_pixel(wcs).to_mask(mode='subpixels')

In [ ]:
data = imviz.get_data(imviz.app.data_collection[0].label)

In [ ]:
plt.imshow(mask.to_image(data.data.shape), origin='lower')

You can use Astrowidgets API.

In [ ]:
viewer = imviz.default_viewer

In [ ]:
# Center the image on given pixel position.
viewer.center_on((6, 5))  # X, Y (0-indexed)

In [ ]:
# Move the image with the given pixel offsets.
viewer.offset_by(5, -3)  # dX, dY

We can convert the coordinates of the interactive subset from pixel to world coordinates like so:

In [ ]:
center = regions['Subset 1'].center

In [ ]:
# Center the image on given sky coordinates.
sky = SkyCoord(ra=269.89, dec=66.04, unit=('deg', 'deg'))
viewer.center_on(sky)

In [ ]:
# Move the image with the given sky offsets.
viewer.offset_by(0.5 * u.arcsec, -1.5 * u.arcsec)

You can programmatically zoom in and out.

Zoom level:

* 1 means real-pixel-size.
* 2 means zoomed in by a factor of 2.
* 0.5 means zoomed out by a factor of 2.
* 'fit' means zoomed to fit the whole image width into display.

In [ ]:
# Get the current zoom level.
viewer.zoom_level

In [ ]:
# Set the zoom level directly.
viewer.zoom_level = 100

In [ ]:
# Set the relative zoom based on current zoom level.
viewer.zoom(1.5)

It is also possible to programmatically add non-interactive markers to the image.

In [ ]:
# Add 20 randomly generated X,Y (0-indexed) w.r.t. reference image
# using default marker properties.
image_shape = data.data.shape
t_xy = Table({'x': np.random.randint(0, image_shape[1], 20),
              'y': np.random.randint(0, image_shape[0], 20)})

viewer.add_markers(t_xy)

You could customize marker color, alpha, size, and fill with values that Glue supports.

In [ ]:
np.random.seed(42)
# Mark some sky coordinates using updated marker properties.
t_sky = Table({'coord': [
    sky.spherical_offsets_by(
         *np.random.normal(scale=0.1, size=2) * u.arcsec
    ) for i in range(5)
]})
viewer.add_markers(t_sky, use_skycoord=True, marker_name='my_sky')
t_sky

When you do not need the markers anymore, you could remove them by name.

In [ ]:
viewer.remove_markers(marker_name='my_sky')

You can also remove all the markers.

In [ ]:
viewer.reset_markers()

You can save the active display as PNG file.

In [ ]:
viewer.save('myimage.png')

If you want a second viewer, run the following cell *or* click on the "picture with +" icon on top of the image viewer.

If you click the icon, the viewer name will be auto-generated. Instead of running the next cell, you can run this instead:

    viewer_2_name = 'imviz-1'
    viewer_2 = imviz.app.get_viewer_by_id(viewer_2_name)

In [ ]:
viewer_2_name = 'my-viewer-1'

# Do not pass in viewer_name if you want it auto-generated.
# If auto-generated, viewer name will be 'imviz-1'.
viewer_2 = imviz.create_image_viewer(viewer_name=viewer_2_name)

You can also programmatically control the new viewer, as follows.

In [ ]:
# Show the first image in the second viewer.
imviz.app.add_data_to_viewer(viewer_2_name, "exposure 1[DATA]")

In [ ]:
viewer_2.zoom(4)

In [ ]:
viewer_2.cuts = '95%'

In [ ]:
viewer_2.center_on((1500, 1000))

To destroy this new viewer, run the following cell *or* click the "x" that is right next to the viewer ID.

In [ ]:
imviz.destroy_viewer(viewer_2_name)